In [1]:
import re
from nltk import tokenize
from nltk.corpus import stopwords
from pymystem3 import Mystem


stem = Mystem()
stopwords_extension = ['руб', 'евро', 'долл', 'млн', 'млрд', 'это', 'вице', 'эль']
reg = re.compile('[А-Яа-яA-Za-z]+')


class ArticleMeta:
    # Initializing URL, publish date, authors, tokenized sentences
    def __init__(self, url, pub_date, authors, title, lead):
        self.url = url
        self.pub_date = pub_date
        self.authors = authors
        self.title = title
        self.lead = lead
        self.sentences, self.tokens = self.lemmatize(lead)
        self.analysis = self.extract_analysis()

    # TODO: SyntaxNet tree
    @staticmethod
    # Preprocess of given lead
    def lemmatize(text):
        tokens = [token for token in re.findall(reg, text.replace('-', '')) if token not in stopwords.words('russian') +
                         stopwords_extension]
        text = ' '.join(tokens)
        lem_text = ''.join(stem.lemmatize(text))
        return lem_text.replace('руб.', 'руб').replace('\xa0', ''), tokens

    def extract_analysis(self):
        string_analysis = ''
        for token in self.tokens:
            if 'analysis' in stem.analyze(token)[0] and len(stem.analyze(token)[0]['analysis']):
                string_analysis += stem.analyze(token)[0]['analysis'][0]['gr']
                string_analysis += ' '
            elif token == '.':
                string_analysis += token
                string_analysis += ' '
        return string_analysis

In [2]:
import pandas as pd
import re
import urllib.request

from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from tqdm import tqdm

# Regular expressions for news lead and publish date
reg_html = re.compile('<[^>]+>')
calendar = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
calendar_leap = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]


# Scraping rubrics and constructing ArticleMeta object
def gen_urls(year):
    urls = []
    for month in range(1): # for speed
        if year in [2017]: # for speed
            if month < 9:
                for day in range(1, calendar_leap[month] + 1):
                    if day < 10:
                        urls.append('https://www.kommersant.ru/archive/rubric/40/' + str(year) + '-0' + str(month + 1) +
                                    '-0' + str(day))
                    else:
                        urls.append('https://www.kommersant.ru/archive/rubric/40/' + str(year) + '-0' + str(month + 1) +
                                    '-' + str(day))
            else:
                for day in range(1, calendar_leap[month] + 1):
                    if day < 10:
                        urls.append('https://www.kommersant.ru/archive/rubric/40/' + str(year) + '-' + str(month + 1) +
                                    '-0' + str(day))
                    else:
                        urls.append('https://www.kommersant.ru/archive/rubric/40/' + str(year) + '-' + str(month + 1) +
                                    '-' + str(day))
        else:
            if month < 9:
                for day in range(1, calendar[month] + 1):
                    if day < 10:
                        urls.append('https://www.kommersant.ru/archive/rubric/40/' + str(year) + '-0' + str(month + 1) +
                                    '-0' + str(day))
                    else:
                        urls.append('https://www.kommersant.ru/archive/rubric/40/' + str(year) + '-0' + str(month + 1) +
                                    '-' + str(day))
            else:
                for day in range(1, calendar[month] + 1):
                    if day < 10:
                        urls.append('https://www.kommersant.ru/archive/rubric/40/' + str(year) + '-' + str(month + 1) +
                                    '-0' + str(day))
                    else:
                        urls.append('https://www.kommersant.ru/archive/rubric/40/' + str(year) + '-' + str(month + 1) +
                                    '-' + str(day))
    return urls

In [10]:
def scrape_archive(year):
    urls = gen_urls(year)
    metas = []
    for url in urls:
        try:
            req = urllib.request.Request(url, headers={'User-Agent': "Magic Browser"})
            html = urllib.request.urlopen(url)
            html = html.read()
            articles = Selector(text=html).xpath('/html/body/div[@class="layout"]/div[@class="col_group"]/'
                                                 'div[@class="col col-large"]/div[@class="col col-big"]/'
                                                 'section[@class="b-other_docs"]/'
                                                 'div[@class="b-other_docs__list"]/a/@href').extract()
            for article in articles:
                url_art = 'https://www.kommersant.ru' + article
                try:
                    req = urllib.request.Request(url_art, headers={'User-Agent': "Magic Browser"})
                    html = urllib.request.urlopen(req).read()
                    html_response = HtmlResponse(url_art, body=html)
                    time = Selector(html_response).xpath('/html/body/div[@class="layout"]/div[@class="col_group"]/'
                                                         'div[@class="col col-large"]/'
                                                         'div[@class="col col-big js-middle"]/article/header/'
                                                         'time/@datetime').extract()[0]
                    title = Selector(html_response).xpath('//body/div[@class="layout"]/div[@class="col_group"]/'
                                                          'div[@class="col col-large"]/'
                                                          'div[@class="col col-big js-middle"]'
                                                          '/article/header/div[@class="text"]/h1').extract()[0]
                    title = re.sub(reg_html, '', title).strip()
                    lead = Selector(html_response).xpath('//body/div[@class="layout"]/div[@class="col_group"]/'
                                                         'div[@class="col col-large"]/'
                                                         'div[@class="col col-big js-middle"]/article/'
                                                         'div[@class="article_text_wrapper"]/'
                                                         'p[@class="b-article__text"]/span').extract()[0]
                    lead = re.sub(reg_html, '', lead).strip()
                    authors = Selector(html_response).xpath('//body/div[@class="layout"]/div[@class="col_group"]/'
                                                            'div[@class="col col-large"]/'
                                                            'div[@class="col col-big js-middle"]/article/'
                                                            'div[@class="article_text_wrapper"]/'
                                                            'p[@class="b-article__text document_authors"]').extract()[0]
                    authors = re.sub(reg_html, '', authors).strip()
                    meta = ArticleMeta(url_art, time, 'Kommersant, ' + authors, title, lead)
                    metas.append(meta)
                except:
                    pass
            print(url + ' is processed')
        except:
            pass
    return metas

In [11]:
# Stack data
def stack_data(metas, year):
    d = {'URL': [], 'Publish Date': [], 'Authors': [], 'Title': [], 'Lead sentences': [],
            'Lemmatized lead sentences': [], 'Lemm Analysis': []}
    print(str(year) + ' ready to be stacked')
    for article in metas:
        d['URL'].append(article.url)
        d['Publish Date'].append(article.pub_date)
        d['Authors'].append(article.authors)
        d['Title'].append(article.title)
        d['Lead sentences'].append(article.lead)
        d['Lemmatized lead sentences'].append(article.sentences)
        d['Lemm Analysis'].append(article.analysis)
    df = pd.DataFrame(data=d)
    df.to_csv('scrape_metadata_' + str(year) + '.csv')
    print(str(year) + ' is stacked into CSV')
    return(df)



def res(year):
    stack_data(scrape_archive(year), year)

data = res(2017)


https://www.kommersant.ru/archive/rubric/40/2017-01-01 is processed
https://www.kommersant.ru/archive/rubric/40/2017-01-02 is processed
https://www.kommersant.ru/archive/rubric/40/2017-01-03 is processed
https://www.kommersant.ru/archive/rubric/40/2017-01-04 is processed
https://www.kommersant.ru/archive/rubric/40/2017-01-05 is processed
https://www.kommersant.ru/archive/rubric/40/2017-01-06 is processed
https://www.kommersant.ru/archive/rubric/40/2017-01-07 is processed
https://www.kommersant.ru/archive/rubric/40/2017-01-08 is processed
https://www.kommersant.ru/archive/rubric/40/2017-01-09 is processed
https://www.kommersant.ru/archive/rubric/40/2017-01-10 is processed
https://www.kommersant.ru/archive/rubric/40/2017-01-11 is processed
https://www.kommersant.ru/archive/rubric/40/2017-01-12 is processed
https://www.kommersant.ru/archive/rubric/40/2017-01-13 is processed
https://www.kommersant.ru/archive/rubric/40/2017-01-14 is processed
https://www.kommersant.ru/archive/rubric/40/2017

In [12]:
data = pd.read_csv('scrape_metadata_2017.csv')

data.head()

,Unnamed: 0,Authors,Lead sentences,Lemm Analysis,Lemmatized lead sentences,Publish Date,Title,URL
0,0,"Kommersant, Юлия Локшина","Вопрос о дальнейшей судьбе банка ""Пересвет"", п...","S,муж,неод=(вин,ед|им,ед) A=(пр,ед,полн,жен|да...",вопрос дальнейший судьба банк пересвет принадл...,2017-01-09T00:00:00+03:00,"Санацию ""Пересвета"" поддержал премьер-министр",https://www.kommersant.ru/doc/3186810
1,1,"Kommersant, Мария Сарычева",Количество сделок слияний и поглощений (M&amp;...,"S,сред,неод=(вин,ед|им,ед) S,жен,неод=род,мн S...",количество сделка слияние поглощение M amp A г...,2017-01-09T00:00:00+03:00,ТЭК попал в аутсайдеры,https://www.kommersant.ru/doc/3186841
2,2,"Kommersant, Виталий Гайдаев",Впервые за четыре года январские валютные торг...,"ADV= NUM=(им|вин,неод) S,муж,неод=(вин,мн|род,...",впервые четыре год январский валютный торги ро...,2017-01-09T00:00:00+03:00,Игра на укрепление российской валюты продолжается,https://www.kommersant.ru/doc/3186807
3,3,"Kommersant, Юлия Полякова, Владислав Новый","ВТБ и АФК ""Система"" договорились о дальнейшей ...","S,сокр,ед,муж,неод=(пр|вин|дат|род|твор|им) S,...",втб АФК система договариваться дальнейший судь...,2017-01-09T00:00:00+03:00,Банк превратился из акционера холдинга в его о...,https://www.kommersant.ru/doc/3186782
4,4,"Kommersant, Евгений Хвостик",Генеральный директор Лондонской фондовой биржи...,"A=(вин,ед,полн,муж,неод|им,ед,полн,муж) S,муж,...",генеральный директор лондонский фондовый биржа...,2017-01-10T21:48:13+03:00,Гендиректор Лондонской фондовой биржи и глава ...,https://www.kommersant.ru/doc/3188014
